In [ ]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

In [ ]:
# Generate summary statistics
df_market_data.describe()

In [ ]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

---

### Prepare the Data

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaled = StandardScaler().fit_transform(df_market_data)
print(scaled[0:3])

In [ ]:
print(df_market_data.columns)

In [ ]:
# Create a DataFrame with the scaled data
df_scaled = pd.DataFrame(scaled, index=df_market_data.index, columns=df_market_data.columns)

# Copy the crypto names from the original data
df_scaled['coin_id'] = df_market_data.index

# Set the coinid column as index
df_scaled.set_index('coin_id', inplace=True)

# Display sample data
df_scaled.head(10)

---

### Find the Best Value for k Using the Original Data.

In [ ]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 12))
print(k_values)

In [ ]:
# Create an empty list to store the inertia values
inertia_values = []


# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list

for k in k_values:
    # Create a KMeans model using the loop counter for the n_clusters
    kmeans = KMeans(n_clusters=k)
    
    # Fit the model to the data using `df_market_data_scaled`
    kmeans.fit(df_scaled)
    
    # Append the model.inertia_ to the inertia list
    inertia_values.append(kmeans.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {
    'k_values': k_values,
    'inertia_values': inertia_values
}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)

df_elbow

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
df_elbow.hvplot.line(
    x="k_values", 
    y="inertia_values", 
    title="Elbow Curve", 
    xlabel='Number of clusters (k)',
    ylabel='Inertia'
)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 4

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [ ]:
# Initialise the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=1)

In [ ]:
# Fit the K-Means model using the scaled data
model.fit(df_scaled)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled data
k_4 = model.predict(df_scaled)

# Print the resulting array of cluster values.
k_4

In [ ]:
# Create a copy of the DataFrame
df_scaled_predictions = df_scaled.copy()

In [ ]:
# Add a new column to the DataFrame with the predicted clusters
df_scaled_predictions['clusters'] = k_4

# Display sample data
df_scaled_predictions.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Colour the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
df_scaled_predictions.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="clusters",
    hover_cols=["coin_id"],
)

---

### Optimise Clusters with Principal Component Analysis.

In [ ]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [ ]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
pca_df = pca.fit_transform(df_scaled)

# View the first five rows of the DataFrame. 
pca_df[:5]

In [ ]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
explained_variance = pca.explained_variance_ratio_
explained_variance

In [ ]:
# **Question:** What is the total explained variance of the three principal components?
total_explained_variance = sum(explained_variance)
total_explained_variance

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 90% 

In [ ]:
# Create a new DataFrame with the PCA data.
scaled_pca_df = pd.DataFrame(
    pca_df,
    columns=["PC1", "PC2","PC3"])

# Copy the crypto names from the original data
scaled_pca_df['coin_id'] = df_market_data.index

# Set the coinid column as index
scaled_pca_df.set_index('coin_id', inplace=True)

# Display sample data
scaled_pca_df.head()

---

### Find the Best Value for k Using the PCA Data

In [ ]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 12))
print(k)

In [ ]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list

for i in k: 
    model = KMeans(n_clusters=i, random_state=0)
    model.fit(scaled_pca_df)
    inertia.append(model.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_pc = {
    "k": k,
    "inertia": inertia
}

# Create a DataFrame with the data to plot the Elbow curve
elbow_pc_df = pd.DataFrame(elbow_pc)
elbow_pc_df

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_pc_df.hvplot.line(
    x="k",
    y="inertia",
    title="PC Elbow Curve",
    xticks=k
)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No, both resulted in the same k value (i.e. 4)

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [ ]:
# Initialise the K-Means model using the best value for k
model = KMeans(n_clusters=4)

In [ ]:
# Fit the K-Means model using the PCA data
model.fit(scaled_pca_df)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the PCA data
prediction_pc = model.predict(scaled_pca_df)
# Print the resulting array of cluster values.
prediction_pc

In [ ]:
# Create a copy of the DataFrame with the PCA data
pca_data_df = scaled_pca_df.copy()

# Add a new column to the DataFrame with the predicted clusters
pca_data_df["Clusters"] = prediction_pc

# Display sample data
pca_data_df.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Colour the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
pca_data_df.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="Clusters",
    hover_cols="coin_id",
    xlabel="price_change_percentage_24h",
    ylabel="price_change_percentage_7d"
)

### Visualise and Compare the Results

In this section, you will visually analyse the cluster analysis results by contrasting the outcome with and without using the optimisation techniques.

In [ ]:
# Composite plot to contrast the Elbow curves
df_elbow.hvplot.line(
    x="k_values", 
    y="inertia_values", 
    title="Elbow Curve", 
    xlabel='Number of clusters (k)',
    ylabel='Inertia'
) + elbow_pc_df.hvplot.line(
    x="k",
    y="inertia",
    title="PC Elbow Curve",
    xticks=k
)

In [ ]:
# Composite plot to contrast the clusters
df_scaled_predictions.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="clusters",
    hover_cols=["coin_id"],
) + pca_data_df.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="Clusters",
    hover_cols="coin_id",
    xlabel="price_change_percentage_24h",
    ylabel="price_change_percentage_7d"
)

#### Answer the following question: 

  * **Question:** After visually analysing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** 